Let's try tuning the prompt for `Qwen/Qwen2.5-0.5B-Instruct`.

In [3]:
from openai import OpenAI 

client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="-"
)

In [4]:
client.models.list()

SyncPage[Model](data=[Model(id='Qwen/Qwen2.5-0.5B-Instruct', created=1756653302, object='model', owned_by='vllm', root='Qwen/Qwen2.5-0.5B-Instruct', parent=None, max_model_len=256, permission=[{'id': 'modelperm-ec9e1c6f62084cdfae956c46286c6f36', 'object': 'model_permission', 'created': 1756653302, 'allow_create_engine': False, 'allow_sampling': True, 'allow_logprobs': True, 'allow_search_indices': False, 'allow_view': True, 'allow_fine_tuning': False, 'organization': '*', 'group': None, 'is_blocking': False}])], object='list')

In [54]:
def create_chat_completion(
    sys_prompt: str, 
    user_msg: str,
    model: str = 'Qwen/Qwen2.5-0.5B-Instruct',
    **kwargs 
):
    res = client.chat.completions.create(
        model=model,
        messages=[
            {
                'role': 'system',
                'content': sys_prompt
            },
            {
                'role': 'user',
                'content': user_msg
            }
        ],
        **kwargs
    )

    return res.choices[0].message.content

Load an exapmle dataset.

In [7]:
from mteb.tasks import NQ

nq = NQ() 
nq.load_data()

/home/oleksii/python/embed_steer/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4201/4201 [00:00<00:00, 40199.01 examples/s]


In [9]:
queries = list(nq.queries['test'].items())
corpus = list(nq.corpus['test'].items())

In [70]:
queries[0][1], corpus[0][1]

('what is non controlling interest on balance sheet',
 "Minority interest In accounting, minority interest (or non-controlling interest) is the portion of a subsidiary corporation's stock that is not owned by the parent corporation. The magnitude of the minority interest in the subsidiary company is generally less than 50% of outstanding shares, or the corporation would generally cease to be a subsidiary of the parent.[1]")

In [71]:
sys_prompt = """
You are a helpful AI assistant. 
Given a document excerpt, generate a concise and highly relevant search question. 
Output only the question in lowercase, without quotes or ending punctuation.
""".strip()

In [72]:
create_chat_completion(
    sys_prompt,
    corpus[0][1], 
    temperature=0
)

'minority interest in accounting'